In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Sources
Ideally, I would like to get data from:
- Reddit episode discussions
    - upvotes
    - number of comments
    - episode ratings through the surveys in episode discussion threads
- IMBD episode ratings
- MAL episode polls

The reddit upvotes and comments are more a measure of popularity than quality. However the episode ratings and polls are likely to be a good measure. Most people will rate episodes they enjoy highly, and will rate low the episodes they don't like. Some people will rate all episodes highly, and some will rate all lowly, that is just the nature of the internet.

In terms of audience, these measures are obviously pretty specific. They are limited to the type of people that participate in these sites, which is a very different audience to the total population that have watched DBZ. Ideally, if one wanted to get a measure of the 'best' DBZ episode with the widest possible sample population, one would include statistics such as TV viewership and media sales. But to keep this project manageable I'll restrict to just the listed data sources. As such, this data will only be representative of the interactive, online, English audience.

---

# Hypotheses
Based off my own internal biases and preconceptions, I reckon we'll get the following:
- Top rated episode is either during Gohan's fight with Perfect Cell, or during the Goku vs Vegeta fight in the Saiyan saga.
- Top episode in each arc is during the final fight, with the exception of the Frieza saga where it's the episode when Goku achieves super saiyan.
- Lowest rated episode will be a filler episode, maybe in the Garlic Jr arc.


# MyAnimeList (MAL)
MAL has an unofficial API that we can use to get data from MAL. First, we can get the episode titles and other info, then hopefully we can use it to get the voting data from the 